<a href="https://colab.research.google.com/github/medeirossteffany/Deteccao_lixos_oceanos/blob/main/Deteccao_lixos_oceanos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
from datetime import datetime
import requests

# Função para obter as coordenadas (latitude e longitude) de um endereço usando a API Nominatim
def obter_coordenadas_nominatim(endereco):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': endereco,
        'format': 'json',
        'limit': 1
    }
    headers = {
        'User-Agent': 'Steffany'  # Altere conforme necessário
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        dados = response.json()
        if dados:
            return float(dados[0]['lat']), float(dados[0]['lon'])
        else:
            return None
    else:
        return None

# Inicializa a lista para armazenar os resultados da análise
resultados = []

# Variável para controlar o loop
encerrar = False

# Pergunta ao usuário se ele já subiu as imagens para o diretório
subiu_imagens = input("Você já subiu as imagens que deseja analisar para o diretório? (sim/não): ").lower()
if subiu_imagens != 'sim':
    print("Por favor, suba as imagens para o diretório antes de continuar.")
else:
    while not encerrar:
        # Pergunta o caminho da imagem ou se deseja finalizar
        caminho_imagem = input("Digite o caminho da imagem para análise, caso deseje finalizar as análises digite 'finalizar': ") # exemplo "marLimpo3.jpg"
        if caminho_imagem.lower() == 'finalizar':
            encerrar = True
        else:
            # Desativa a notação científica para clareza
            np.set_printoptions(suppress=True)

            # Carrega o modelo
            modelo = load_model("keras_model.h5", compile=False)

            # Carrega os rótulos
            nomes_classes = ['Mar Limpo', 'Mar Sujo']

            # Cria o array do formato correto para alimentar o modelo keras
            dados = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

            # Abre a imagem e converte para RGB
            imagem = Image.open(caminho_imagem).convert("RGB")

            # Redimensiona a imagem para pelo menos 224x224 e recorta do centro
            tamanho = (224, 224)
            imagem = ImageOps.fit(imagem, tamanho, Image.Resampling.LANCZOS)

            # Converte a imagem em um array numpy
            array_imagem = np.asarray(imagem)

            # Normaliza a imagem
            array_imagem_normalizada = (array_imagem.astype(np.float32) / 127.5) - 1

            # Carrega a imagem no array
            dados[0] = array_imagem_normalizada

            # Faz a predição do modelo
            previsao = modelo.predict(dados)
            indice = np.argmax(previsao)
            nome_classe = nomes_classes[indice].strip()
            confianca = previsao[0][indice]

            # Obtém a data e hora atual
            data_hora_atual = datetime.now()

            # Pergunta o endereço do usuário
            endereco_usuario = input("Digite o endereço da sua localização: ")

            # Obtém as coordenadas do endereço usando if e else
            coordenadas = obter_coordenadas_nominatim(endereco_usuario)
            if coordenadas is not None:
                lat, lng = coordenadas
                print(f'Data e hora: {data_hora_atual}')
                print(f'Latitude: {lat}, Longitude: {lng}')
            else:
                print("Erro ao obter as coordenadas")
                lat, lng = None, None

            # Imprime a predição e a confiança
            print("A imagem é:", nome_classe)
            print("Confiança:", confianca)

            # Armazena o resultado na lista
            resultados.append({
                "data_hora": data_hora_atual,
                "nome_classe": nome_classe,
                "confianca": confianca,
                "endereco": endereco_usuario,
                "latitude": lat,
                "longitude": lng
            })

    # Após o loop terminar, permite ao usuário revisar os resultados
    for i, resultado in enumerate(resultados):
        print(f"Análise {i + 1}:")
        print(f"Data e hora: {resultado['data_hora']}")
        print(f"Classificação: {resultado['nome_classe']}")
        print(f"Confiança: {resultado['confianca']}")
        print(f"Endereço: {resultado['endereco']}")
        print(f"Latitude: {resultado['latitude']}, Longitude: {resultado['longitude']}")



Você já subiu as imagens que deseja analisar para o diretório? (sim/não): sim
Digite o caminho da imagem para análise, caso deseje finalizar as análises digite 'finalizar': marLimpo3.jpg
1/1 [==============================] - 1s 968ms/step
Digite o endereço da sua localização: avenida paulista
Data e hora: 2024-06-03 17:03:13.713754
Latitude: -23.5577504, Longitude: -46.6604517
A imagem é: Mar Limpo
Confiança: 0.9999865
Digite o caminho da imagem para análise, caso deseje finalizar as análises digite 'finalizar': finalizar
Análise 1:
Data e hora: 2024-06-03 17:03:13.713754
Classificação: Mar Limpo
Confiança: 0.9999865293502808
Endereço: avenida paulista
Latitude: -23.5577504, Longitude: -46.6604517
